In [26]:
# Import required libraries
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

In [27]:
# NOTE: only using Apple data at the moment
# unsure if its better to train per stock or all at once

# Create dataframes for the tweets and prices 
tweet_df = pd.read_csv('csv/aapl.csv')
price_df = pd.read_csv('csv/aapl_price.csv') # Retrieved from https://www.nasdaq.com/market-activity/stocks/aapl/historical

# Rename the date column so they match between dataframes
tweet_df.rename(columns = {'DATE': 'date'}, inplace = True)
price_df.rename(columns = {'Date' : 'date'}, inplace = True)

# Merge the dataframes based on the date column
merged_df = pd.merge(tweet_df, price_df, on = 'date')

# Save to csv for double checking
#merged_df.to_csv('merged.csv', index = False)

In [28]:
# Rename the sentiment score and stock price to something more maliable
merged_df.rename(columns = {'TEXTBLOB_POLARITY': 'sentimentScore', 'Close/Last': 'stockPrice'}, inplace = True)

# Create the final dataframe with only the sentiment score and stock price
final_df = merged_df[['sentimentScore', 'stockPrice']]

# Remove the $ sign in stock price, and convert both columns to float64 (may be redundant)
final_df.loc[:, 'stockPrice'] = final_df['stockPrice'].str.replace('$', '')
final_df.loc[:, 'sentimentScore'] = pd.to_numeric(final_df['sentimentScore'])
final_df.loc[:, 'stockPrice'] = pd.to_numeric(final_df['stockPrice'])

In [29]:
# NOTE: this scales the values for supposedly better allignment, but may not give intended results 

# Create the scaler
scaler = MinMaxScaler()

# Scale both values 
final_df.loc[:, ['sentimentScore', 'stockPrice']] = scaler.fit_transform(final_df[['sentimentScore', 'stockPrice']])


In [30]:
# Split the dataframe into X and y
X = final_df['sentimentScore'].values
y = final_df['stockPrice'].values

In [31]:
# NOTE: reshaping needs to be done properly according to accurate LSTM inputs
# sorry, first time using tensorflow and LSTMs
# see https://towardsdatascience.com/implementation-differences-in-lstm-layers-tensorflow-vs-pytorch-77a31d742f74

# Reshape X
X = X.reshape(-1, 1, 1)

# Split X and Y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# More reshaping that needs to be fixed
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [32]:
# NOTE: model can be changed
# right now there's 3 layers of LSTM and Dropout however better results may be achived with different LSTM layering

# LSTM Model with Dropout and Dense
model = Sequential([
    Input(shape = X_train.shape[1:]),
    LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1:], 1)),
    Dropout(0.2), 
    LSTM(units = 50, return_sequences = True),
    Dropout(0.2),
    LSTM(units = 50),
    Dropout(0.2),
    Dense(units = 1)
])

# Use the Adam optimizer
model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'mean_sqaured_error')

c:\Users\Main\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [33]:
# NOTE: does not work due to improper shaping
# see above link in NOTES

# Fit the model with verbose settings for debugging
model.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose = 1)

# Calculate and print the loss
loss = model.evaluate(X_test, y_test)
print("Test Loss: ", loss)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).